# Load Data

In [1]:
import music21
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import mnet

In [ ]:
'''
Creat local corpus with access pieces
'''

localCorpus = music21.corpus.corpora.LocalCorpus()
localCorpus.addPath('../library')
music21.corpus.cacheMetadata()

s = music21.corpus.parse('telemannfantasie1.xml')
chord_lst = list(s.chordify().recurse().notes)
nodelst = mnet.convert_chord_note(chord_lst, 'A')
g_rn=mnet.create_graph(nodelst)

#write as .gexf
#nx.write_gexf(g_rn, "rn_composition_notechord.gexf")


Uses Full Roman Numeral graph

## Clustering

### InfoMap

Write to pajek

In [10]:
g_weighted=mnet.convert_to_weighted(g_rn, fraction=False)
nx.write_pajek(g_weighted, "../graphs/rn.net")

Copy weighted graph

In [11]:
g_info = g_weighted.copy()

Assign 'biggroup' and 'smallgroup' 
.net.tree file generated from Infomap web app

In [12]:
bigDict = {}
smallDict = {}
with open("../graphs/rn.net.tree") as tree:
    for line in tree:
        spl = line.split('"')
        node = spl[1]
        path = spl[0].split(':')
        bigGroup=path[0]
        smallGroup=path[0]+path[1]
        #print(node, bigGroup, smallGroup)
        bigDict[node]=bigGroup
        smallDict[node]=smallGroup
        
        
        

In [13]:
nx.set_node_attributes(g_info, bigDict, 'bigGroup')
nx.set_node_attributes(g_info, smallDict, 'smallGroup')

Read this next time

In [14]:
nx.write_gexf(g_info, "../graphs/rn_info.gexf")

In [15]:
g_info.nodes

NodeView(('A4 I', 'B4 I', 'E4 I', 'C#5 I', 'E5 I', 'A5 I', 'D5 IV64', 'D5 I', 'E5 V', 'G4 I', 'C#6 I', 'A3 I', 'G#4 I', 'B4 V65', 'F#5 IV64', 'end', 'D4 IV64', 'F#4 I', 'D#5 I', 'F#5 vi6', 'F#5 I', 'E4 V6', 'G#4 V6', 'G#5 I', 'C#5 vi6', 'G#4 iii64', 'A4 IV64', 'C#5 IV64', 'A5 IV64', 'E5 IV64', 'B4 IV64', 'D5 ii', 'G#5 IV64', 'B5 IV64', 'D#4 #ivob64', 'C#5 #ivob64', 'B4 #ivob64', 'A4 #ivob64', 'A5 #ivob64', 'D5 V6', 'E3 vii', 'B4 V6', 'E5 V6', 'G#5 V6', 'D#5 V6', 'B5 V6', 'F#4 II#643', 'F#4 II75#3', 'A5 vi6', 'B4 vi6', 'G#5 vi6', 'E5 vi6', 'B4 ii', 'F#5 ii', 'C#5 ii', 'G#4 V43', 'E5 V65', 'E4 V65', 'D5 V65', 'G#4 V65', 'G#4 V', 'A5 V', 'C#5 V', 'F#4 V', 'G#5 iii6', 'E4 iii6', 'C#6 iii6', 'A4 iii6', 'A5 vi', 'F#5 vi', 'B5 II#643', 'B4 II#643', 'B4 V', 'A#4 VI75#3', 'E5 VI75#3', 'F#5 VI75#3', 'D5 ii64', 'G5 VI75#3', 'F#4 ii64', 'B4 ii64', 'A4 ii64', 'G#4 vii/o7', 'D5 vii/o7', 'E5 vii/o7', 'F#5 vii/o7', 'D5 IV', 'F#4 IV', 'C#5 IV', 'F#5 IV', 'D4 IV', 'A5 IV', 'A4 I6b5', 'E5 IV', 'B5 II', '

Generate sets of groups

In [43]:
g_info

In [216]:
def generateCommunities(group_dict, graph):
    communityLookup = {}
    for node in group_dict.keys():
        val = group_dict[node]
        if val not in communityLookup.keys():
            communityLookup[val]=set([])
        communityLookup[val].add(node)
    
    subgraphlst=[]
    for label in communityLookup.keys():
        subgraph = graph.subgraph(communityLookup[label])
        subgraphlst.append(subgraph)
    return communityLookup, subgraphlst
        
    
        

In [220]:
sglst1, c1 = generateSubgraphs(bigDict, g_info)

In [221]:
sglst2, c2 = generateSubgraphs(smallDict, g_info)

## Find Connections between graphs

In [110]:
'''
pass in blank list
'''
def findpred(graph, edge, lst):
    #print(list(graph.neighbors(edge[0])))    
    
    #the only neighbor, i.e. when node is reached 100% probability of switching communities
    if len(list(graph.neighbors(edge[0]))) == 1:     

        for pred in list(graph.predecessors(edge[0])):
           findpred(graph, (pred, edge[0]), lst)
    else:
           lst.append(edge)

In [111]:
def findBridges(nodeset1, nodeset2):
    bigcom1 = g_info.subgraph(nodeset1)
    bigcom2 = g_info.subgraph(nodeset2)
    bigcom12 = g_info.subgraph(nodeset1  | nodeset2)
    edges = bigcom12.edges- bigcom1.edges-bigcom2.edges
    edgelst1 = []
    edgelst2 = []
    

               
    for edge in edges:
        if edge[0] in nodeset1:
            findpred(bigcom12, edge, edgelst1)
        else:
            findpred(bigcom12, edge, edgelst2)
    return edgelst1, edgelst2

In [112]:
bridge12, bridge21 = findBridges(c1['1'], c1['2'])

In [113]:
bridge13, bridge31 = findBridges(c1['1'], c1['3'])

In [114]:
findBridges(c1['2'], c1['3'])

([], [])

# Strengthen Edges to Communities 2 and 3

See fraction of traffic leaving each node

In [115]:
g_fraction=mnet.convert_to_weighted(g_rn, fraction= True)

In [116]:
for el in bridge12:
    print(el, g_fraction.get_edge_data(el[0], el[1])['weight'])

('G4 I', 'D5 IV') 0.5
('A4 Ib753', 'G3 I') 0.3333333333333333


In [117]:
for el in bridge13:
    print(el, g_fraction.get_edge_data(el[0], el[1])['weight'])    

('A4 I', 'E5 V') 0.020833333333333332


### Currently the transition into Community 3 is very unlikely (Will only transition 2% of the time. I will use mnet.degree_increase to increase that probability to 30%

In [191]:
g_rn1 = g_rn.copy()

In [192]:
g_rn1 = mnet.degree_increase(g_rn1, bridge13, 15)

In [193]:
g_rn1 = mnet.degree_increase(g_rn1, bridge12, 10)

In [194]:
g_fraction1=mnet.convert_to_weighted(g_rn1, fraction= True)

In [195]:
for el in bridge13:
    print(el, g_fraction1.get_edge_data(el[0], el[1])['weight'])   

('A4 I', 'E5 V') 0.25396825396825395


In [196]:
for el in bridge12:
    print(el, g_fraction1.get_edge_data(el[0], el[1])['weight'])  

('G4 I', 'D5 IV') 0.9166666666666666
('A4 Ib753', 'G3 I') 0.8461538461538461


Observe change in PageRank

In [197]:
nx.write_gexf(g_rn, "../graphs/rn_communitychange1.gexf")

Generate Labeled music

In [198]:
def str_rn_full_annotated(randomwalk, nodeToGroup_dict):
    harmlst=[]
    mellst = []
    lyriclst = []
    for node in randomwalk:

        mel = node.split()[0]
        n = music21.note.Note(mel)
        n.duration.quarterLength =.5
        lyric= nodeToGroup_dict[node]
        n.lyric = lyric
        lyriclst.append(lyric)
        mellst.append(n)
        
  
    return mellst, lyriclst


In [199]:
randomwalk=mnet.generate_randomwalk(g_rn1)
melody_list, harmony_list = str_rn_full_annotated(randomwalk, bigDict)
melody_stream = mnet.convert_to_stream(randomwalk, melody_list)



In [200]:
randomwalk

['A4 I',
 'E5 V',
 'G#4 V',
 'A5 V',
 'F#4 V',
 'G#5 iii6',
 'E4 iii6',
 'C#6 iii6',
 'A4 iii6',
 'A5 vi',
 'F#5 vi',
 'B5 II#643',
 'B4 II#643',
 'E5 V',
 'G#4 V',
 'C#5 V',
 'B4 V',
 'A#4 VI75#3',
 'E5 VI75#3',
 'D5 ii64',
 'F#4 ii64',
 'B4 ii64',
 'A4 ii64',
 'G#4 vii/o7',
 'D5 vii/o7',
 'C#5 I',
 'E5 I',
 'C#5 I',
 'E5 I',
 'A4 I',
 'E5 V',
 'G#4 V',
 'C#5 V',
 'B4 V',
 'A#4 VI75#3',
 'E5 VI75#3',
 'F#5 VI75#3',
 'E5 VI75#3',
 'F#5 VI75#3',
 'E5 VI75#3',
 'D5 ii64',
 'F#4 ii64',
 'B4 ii64',
 'A4 ii64',
 'G#4 vii/o7',
 'D5 vii/o7',
 'E5 vii/o7',
 'F#5 vii/o7',
 'E5 vii/o7',
 'D5 vii/o7',
 'C#5 I',
 'D4 IV64',
 'B5 IV64',
 'E5 V7',
 'G#5 V7',
 'A4 I',
 'G#4 I',
 'A4 I',
 'E4 I',
 'A4 I',
 'C#5 I',
 'A4 I',
 'C#5 I',
 'D#5 I',
 'E5 I',
 'C#5 I',
 'A5 I',
 'E5 I',
 'C#5 I',
 'A4 I',
 'E5 V',
 'G#4 V',
 'C#5 V',
 'B4 V',
 'A#4 VI75#3',
 'E5 VI75#3',
 'D5 ii64',
 'F#4 ii64',
 'B4 ii64',
 'A4 ii64',
 'G#4 vii/o7',
 'D5 vii/o7',
 'C#5 I',
 'A5 I',
 'C#5 I',
 'A4 I',
 'E5 V',
 'G#4 V',
 'C#

In [201]:

#Write to MIDI 
file = melody_stream.write('xml', "rn_meladjust1.xml")


### Measure Random Walk group composition

In [202]:
g1=0
g2=0
g3=0
for el in randomwalk:
    if el in c1['1']:
        g1 += 1
    if el in c1['2']:
        g2 += 1    
    if el in c1['3']:
        g3 += 1    
        

In [203]:
g1, g2, g3

(390, 43, 138)

## Strengthen Sequences

In [240]:
g_rn2 = g_rn.copy()

Sequences usually have 1 less edge than nodes

In [241]:
for subgraph in sglst2:
    if len(list(subgraph.nodes())) - 1 == len(list(subgraph.edges())):
        for node in list(subgraph.nodes):
            if len(list(subgraph.predecessors(node))) == 0: #entrance node
                preds = g_rn2.predecessors(node)
                prededgelst=[]
                for p in preds:
                    prededgelst.append((p, node))
                g_rn2 = mnet.degree_increase(g_rn2, prededgelst, 2)
                    
                
        

In [258]:
nx.write_gexf(g_rn2, "../graphs/rn_communitychange2.gexf")

In [243]:
randomwalk=mnet.generate_randomwalk(g_rn2)


In [244]:
melody_list, harmony_list = str_rn_full_annotated(randomwalk, smallDict)
melody_stream = mnet.convert_to_stream(randomwalk, melody_list)


In [245]:
randomwalk

['A4 I',
 'B4 V65',
 'E4 V65',
 'G#4 V65',
 'A4 I',
 'B4 I',
 'A4 I',
 'E4 I',
 'A4 I',
 'A5 I',
 'F#5 IV64',
 'D5 IV64',
 'C#5 I',
 'D5 I',
 'E5 I',
 'C#5 I',
 'E5 I',
 'F#5 I',
 'G#5 I',
 'A5 I',
 'E5 I',
 'F#5 I',
 'D5 I',
 'C#5 I',
 'B4 I',
 'C#5 I',
 'D5 I',
 'C#5 I',
 'A4 I',
 'G4 I',
 'D5 IV',
 'F#4 IV',
 'G5 IV',
 'E4 IV',
 'F#5 ii6',
 'D4 ii6',
 'B5 ii6',
 'G4 ii6',
 'G5 v',
 'E5 v',
 'A5 I64b3',
 'A4 I64b3',
 'D4 IV',
 'D5 IV',
 'A5 IV',
 'C#5 IV',
 'D5 IV',
 'C#5 IV',
 'D5 IV',
 'D4 IV',
 'D5 IV',
 'C#5 IV',
 'D5 IV',
 'A5 IV',
 'C#5 IV',
 'D5 IV',
 'C#5 IV',
 'D5 IV',
 'E5 IV',
 'C#5 IV',
 'D5 IV',
 'E5 IV',
 'C#5 IV',
 'D5 IV',
 'C#5 IV',
 'D5 IV',
 'C#5 IV',
 'D5 IV',
 'F#4 IV',
 'G5 IV',
 'E4 IV',
 'F#5 ii6',
 'D4 ii6',
 'B5 ii6',
 'G4 ii6',
 'G5 v',
 'E5 v',
 'A5 I64b3',
 'A4 I64b3',
 'D4 IV',
 'D5 IV',
 'C#5 IV',
 'D5 IV',
 'A5 IV',
 'C#5 IV',
 'D5 IV',
 'A5 IV',
 'D5 IV',
 'D5 IV',
 'F#5 IV',
 'D5 IV',
 'F#4 IV',
 'G5 IV',
 'E4 IV',
 'F#5 ii6',
 'D4 ii6',
 'B5 ii6',
 

In [246]:

#Write to MIDI 
file = melody_stream.write('xml', "rn_meladjust2.xml")


## Strengthen Non - Sequences

In [268]:
g_rn3 = g_rn.copy()

In [269]:
for subgraph in sglst2:
    if len(list(subgraph.nodes())) - 1 != len(list(subgraph.edges())):
        print("found")
        for node in list(subgraph.nodes):

            if len(list(subgraph.predecessors(node))) != len(list(g_rn3.predecessors(node))): #entrance node
                preds = set(g_rn3.predecessors(node)) - set(subgraph.predecessors(node))
                prededgelst=[]
                for p in preds:
                    prededgelst.append((p, node))
                g_rn3 = mnet.degree_increase(g_rn3, prededgelst, 2)
                print("edges added")
                print(prededgelst)
                    
                
        

found
edges added
[('E5 V6', 'E4 I')]
edges added
[('D5 V65', 'D5 I')]
edges added
[('E4 V6', 'C#5 I'), ('D5 vii/o7', 'C#5 I'), ('F#5 IV64', 'C#5 I'), ('E3 vii', 'C#5 I'), ('D5 IV64', 'C#5 I'), ('B4 IV64', 'C#5 I')]
edges added
[('D5 V6', 'A4 I'), ('E4 V6', 'A4 I'), ('G#5 V7', 'A4 I'), ('G#4 V65', 'A4 I'), ('E4 V65', 'A4 I'), ('A5 IV64', 'A4 I'), ('E5 V64', 'A4 I'), ('D5 V43', 'A4 I')]
found
edges added
[('A5 I', 'A4 IV64')]
edges added
[('A5 I', 'F#5 IV64'), ('A4 I', 'F#5 IV64'), ('A5 vi6', 'F#5 IV64')]
edges added
[('A4 I', 'D5 IV64'), ('E3 vii', 'D5 IV64'), ('G5 Ib753', 'D5 IV64')]
edges added
[('D4 IV64', 'C#5 IV64')]
found
edges added
[('E5 I', 'E4 V6'), ('D#5 II75#3', 'E4 V6'), ('A4 #ivob64', 'E4 V6'), ('E5 II#643', 'E4 V6')]
edges added
[('E5 I', 'G#4 V6'), ('A4 II', 'G#4 V6')]
edges added
[('D#5 II75#3', 'E5 V6'), ('B4 ii', 'E5 V6'), ('D5 ii', 'E5 V6')]
found
edges added
[('D5 IV', 'B5 II')]
edges added
[('E5 vi6', 'D#5 II')]
found
edges added
[('D5 ii', 'G#4 V43'), ('C#5 I6b5'

In [270]:
nx.write_gexf(g_rn3, "../graphs/rn_communitychange3.gexf")

In [271]:
randomwalk=mnet.generate_randomwalk(g_rn3)

In [272]:
melody_list, harmony_list = str_rn_full_annotated(randomwalk, smallDict)
melody_stream = mnet.convert_to_stream(randomwalk, melody_list)

In [273]:

#Write to MIDI 
file = melody_stream.write('xml', "../xml/rn_meladjust3.xml")


## Strengthen Large Community Shift + Non-Sequences

In [266]:
g_rn4 = g_rn.copy()

In [267]:
g_rn4 = mnet.degree_increase(g_rn4, bridge13, 15)
g_rn4 = mnet.degree_increase(g_rn4, bridge12, 10)

In [274]:
for subgraph in sglst2:
    if len(list(subgraph.nodes())) - 1 != len(list(subgraph.edges())):
        print("found")
        for node in list(subgraph.nodes):

            if len(list(subgraph.predecessors(node))) != len(list(g_rn4.predecessors(node))): #entrance node
                preds = set(g_rn4.predecessors(node)) - set(subgraph.predecessors(node))
                prededgelst=[]
                for p in preds:
                    prededgelst.append((p, node))
                g_rn2 = mnet.degree_increase(g_rn4, prededgelst, 2)
                print("edges added")
                print(prededgelst)
                    
                
        

found
edges added
[('E5 V6', 'E4 I')]
edges added
[('D5 V65', 'D5 I')]
edges added
[('E4 V6', 'C#5 I'), ('D5 vii/o7', 'C#5 I'), ('F#5 IV64', 'C#5 I'), ('E3 vii', 'C#5 I'), ('D5 IV64', 'C#5 I'), ('B4 IV64', 'C#5 I')]
edges added
[('D5 V6', 'A4 I'), ('E4 V6', 'A4 I'), ('G#5 V7', 'A4 I'), ('G#4 V65', 'A4 I'), ('E4 V65', 'A4 I'), ('A5 IV64', 'A4 I'), ('E5 V64', 'A4 I'), ('D5 V43', 'A4 I')]
found
edges added
[('A5 I', 'A4 IV64')]
edges added
[('A5 I', 'F#5 IV64'), ('A4 I', 'F#5 IV64'), ('A5 vi6', 'F#5 IV64')]
edges added
[('A4 I', 'D5 IV64'), ('E3 vii', 'D5 IV64'), ('G5 Ib753', 'D5 IV64')]
edges added
[('D4 IV64', 'C#5 IV64')]
found
edges added
[('E5 I', 'E4 V6'), ('D#5 II75#3', 'E4 V6'), ('A4 #ivob64', 'E4 V6'), ('E5 II#643', 'E4 V6')]
edges added
[('E5 I', 'G#4 V6'), ('A4 II', 'G#4 V6')]
edges added
[('D#5 II75#3', 'E5 V6'), ('B4 ii', 'E5 V6'), ('D5 ii', 'E5 V6')]
found
edges added
[('D5 IV', 'B5 II')]
edges added
[('E5 vi6', 'D#5 II')]
found
edges added
[('D5 ii', 'G#4 V43'), ('C#5 I6b5'

In [275]:
nx.write_gexf(g_rn3, "../graphs/rn_communitychange4.gexf")

In [276]:
randomwalk=mnet.generate_randomwalk(g_rn4)

In [277]:
melody_list, harmony_list = str_rn_full_annotated(randomwalk, smallDict)
melody_stream = mnet.convert_to_stream(randomwalk, melody_list)

In [278]:

#Write to MIDI 
file = melody_stream.write('xml', "../xml/rn_meladjust4.xml")


## Increase Probability of Staying in Level 2 Community

In [279]:
g_rn5 = g_rn.copy()

In [280]:
for subgraph in sglst2:
    g_rn5 = mnet.degree_increase(g_rn5, list(subgraph.edges()), 4)


In [281]:
nx.write_gexf(g_rn3, "../graphs/rn_communitychange5.gexf")

In [286]:
randomwalk=mnet.generate_randomwalk(g_rn5)

In [287]:
melody_list, harmony_list = str_rn_full_annotated(randomwalk, smallDict)
melody_stream = mnet.convert_to_stream(randomwalk, melody_list)

In [288]:

#Write to MIDI 
melody_stream.write('xml', "../xml/rn_meladjust5.xml")


'/Users/kaitlinpet/Desktop/Coursework/networks/project/xml/rn_meladjust5.xml'